# Redes neuronales para análisis de sentimiento de reseñas de películas

En este notebook exploraremos un dataset formado por reseñas de IMDB dejadas por usuarios sobre determinadas películas y el sentimiento que estas transmiten (positivo o negativo). Estaremos por tanto realizando un trabajo de clasificación binaria siendo nuestras dos clases reseñas positivas o negativas.

Emplearemos los siguientes módulos:

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

Comenzamos cargando los datos a partir del archivo csv correspondiente:

In [ ]:
raw_data = pd.read_csv('./data/movies.csv')

Comprobamos que los datos se hayan cargado correctamente:

In [ ]:
raw_data.head()

In [ ]:
raw_data.shape

Estamos trabajando con 50.000 filas, es decir, con 50.000 opiniones y dos variables, las opiniones y su sentimiento. Procedemos a realizar un brevísimo análisis exploratorio:

## Análisis Exploratorio

### Diccionario de datos

En este caso contamos solo con dos variables:

* La opinión sobre la película **review** que emite una opinión sobre la misma.
* La intención del comentario **sentiment** en la que distinguimos solo entre opiniones positivas y negativas.

Procedemos a continuación a explorar brevemente estas dos variables. Lo cierto es que sobre las opiniones solo podemos comprobar si hay huecos o no.

### Datos perdidos

Comprobamos si existen datos perdidos:

In [ ]:
raw_data.info()

En principio los datos se encuentran completos.

### Análisis de sentiment

Estudiamos la variable **sentiment**:

In [ ]:
raw_data.sentiment.value_counts()

In [ ]:
sns.countplot(x='sentiment', data=raw_data);

Observamos que nos enfrentamos a una clasificación binaria en la que las clases se encuentran totalmente equilibradas. Tenemos 25000 reseñas positivas y 25000 reseñas negativas.

Hasta aquí tenemos nuestro análisis exploratorio. Por tratarse de dato no estructurado y una variable etiqueta realmente no se puede profundizar mucho más. Procedemos a continuación al preprocesado de variables.

## Preprocesamiento de variables

El análisis de sentimiento requiere un preprocesamiento bastante específico:

* Respecto a los textos debemos eliminar las palabras más comunes que no aportan información (conjunciones, preposiciones, pronombres) y tokenizar (convertir nuestras palabras en números).

* Respecto a la etiqueta (positiva o negativo) debemos convertirla en una etiqueta numérica:

### Codificando la etiqueta

Codificaremos los positivos como 1 y los negativos como 0:

In [ ]:
etiquetas = np.array([1 if s == 'positive' else 0 for s in raw_data.sentiment])

In [ ]:
etiquetas

Almacenamos las reseñas en un array:

In [ ]:
reviews = raw_data.review.to_numpy()

In [ ]:
reviews

### Separamos en entrenamiento y validación

Como ya estamos habituado separamos en dos conjuntos: uno de entrenamiento y uno de validación.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    reviews,
    etiquetas,
    test_size=0.2,
    stratify=etiquetas,
    random_state=42,
)

### Preprocesamiento del texto

Comenzamos tokenizando nuestras reseñas. Recordemos que las redes neuronales solo admiten vectores numéricos como entrada por lo que es necesario convertir nuestras palabras en números y nuestras frases en vectores para poder entrenar una red neuronal. Keras nos proporcioa ya un tokenizador que solo requiere indicarle el tamaño de vocabulario, es decir, cuántas palabras distintas lo van a formar:

In [ ]:
SIZE_VOCAB = 10000

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=SIZE_VOCAB)
tokenizer.fit_on_texts(X_train)

Definimos la función seq_pad que nos permite por una parte tokenizar las reseñas y por otra homogeneizar su longitud, es decir, hacer que todas las reseñas tengan el mismo número de palabras, para ello simplemente se fija un número máximo de palabras, si la reseña tiene más se corta y si tiene menos se rellena con 0's hasta alcanzar dicho número

In [ ]:
def seq_pad(raw_texts, tokenizer, max_seq_len=MAX_SEQ_LEN):
    seq = tokenizer.texts_to_sequences(raw_texts)
    pad_seq = keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_seq_len, padding='post')
    return pad_seq

## Construcción del modelo

Podemos proceder ya a construir nuestra red neuronal. Comenzamos usando un Embedding que nos permite crear un espacio vectorial de palabras, posteriormente usamos una red recurrente que permite que la red tenga en cuenta el orden de las palabras para sacar conclusiones y posteriormente colocamos una capa con 16 neuronas y una última capa de 1 neurona. La última capa solo tiene una neurona porque para cada secuencia solo nos interesa un valor: la probabilidad de que la opinión sea positiva:

In [ ]:
EMBED_DIM = 128
MAX_SEQ_LEN = 100

In [ ]:
from tensorflow.keras import layers
model = keras.Sequential()

model.add(layers.Embedding(input_dim=SIZE_VOCAB,output_dim=EMBED_DIM,input_length=MAX_SEQ_LEN))
model.add(layers.SimpleRNN(12))
model.add(layers.Dense(16))
model.add(layers.Dense(1))  # Una sola neurona pues nos interesa un solo valor

model.summary()

Una vez construido el modelo lo compilamos eligiendo nuestro optimizador, la función de pérdida y la métrica adecuada:

In [ ]:
model.compile(
    optimizer="adamax",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

Una vez hecho esto procedemos al entrenamiento del modelo:

In [ ]:
model.fit(
    x=seq_pad(X_train, tokenizer),
    y=y_train,
    batch_size=32,
    epochs=10,
    validation_data=(seq_pad(X_val, tokenizer), y_val),
)

Ya tenemos nuestro modelo. Definimos una función que nos permita realizar predicciones a partir del mismo. Para ello definimos la siguiente función:

In [ ]:
def predict(reviews, model):
    pred = model.predict(seq_pad(reviews, tokenizer))  #tokenizamos la frase y predecimos la probabilidad
    pred_senti = ['positive' if p >= 0.5 else 'negative' for p in pred] # redondeamos a positiva si la probabilidad es 0.5 o más

    for tr, s in zip(reviews, pred_senti):
        print(tr, '-->', s)

Podemos comenzar con algunas reseñas bastante sencillas y observamos que acierta:

In [ ]:
nuevas_reviews = ['I totally loved the film', 'Titanic was ok but the ending ruined it', 'It was boring as hell', 'Terrible']

In [ ]:
predict(nuevas_reviews, model)

Veamos como gestiona algunas más complicadas:

In [ ]:
reviews_complicadas = ['Thanks for wasting two hours of my life', 'I laughed a lot. It was actually a drama.', 'This movie is so good that words cannot express how amazing it is.']


In [ ]:
predict(reviews_complicadas, model)

En un principio atina bastante aunque evidentemente sigue teniendo errores pues solo acierta un 85% en validación. Te propongo que intentes engañarlo tu mismo:

Podeís probar a introducir vuestras propias reviews y visualizar los resultados de los modelos.

In [ ]:
predict(['Prueba a engañarle'], model)

Con esto llegamos al final del notebook en el que con apenas 30 celdas hemos construido una red neuronal capaz de interpretar las reseñas de personas con una precisión bastante alta. Si intentáis jugar con los parámetros de la red aumentando las neuronas y la venta quizá esta precisión sería incluso mejorable. No dudéis en interactuar con ella.